# Banking77 Dataset Exploration

This notebook contains exploratory data analysis (EDA) for the Banking77 dataset.


In [ ]:
import sys
sys.path.append('../src')

from data.load_data import load_banking77
from data.label_mapping import get_label_mappings


## Load the Dataset


In [ ]:
# Load datasets
train, validation, test = load_banking77()


## Exploratory Data Analysis

Add your EDA cells here...
